In [5]:
# First cell - clean everything
!pip uninstall torch torchtext -y
!pip cache purge

Files removed: 6 (188.8 MB)


In [7]:
!pip install torch==2.0.0 torchtext==0.15.0

   ---------------------------------------- 0.0/172.3 MB ? eta -:--:--
   ---------------------------------------- 0.5/172.3 MB 4.2 MB/s eta 0:00:41
   ---------------------------------------- 1.0/172.3 MB 3.9 MB/s eta 0:00:45
   ---------------------------------------- 1.8/172.3 MB 3.1 MB/s eta 0:00:55
    --------------------------------------- 2.6/172.3 MB 3.4 MB/s eta 0:00:50
    --------------------------------------- 3.4/172.3 MB 3.5 MB/s eta 0:00:49
    --------------------------------------- 3.9/172.3 MB 3.6 MB/s eta 0:00:48
   - -------------------------------------- 5.0/172.3 MB 3.6 MB/s eta 0:00:48
   - -------------------------------------- 5.8/172.3 MB 3.6 MB/s eta 0:00:47
   - -------------------------------------- 6.6/172.3 MB 3.6 MB/s eta 0:00:47
   - -------------------------------------- 7.3/172.3 MB 3.6 MB/s eta 0:00:46
   - -------------------------------------- 8.1/172.3 MB 3.6 MB/s eta 0:00:46
   -- ------------------------------------- 8.9/172.3 MB 3.6 MB/s eta 0

Reason for being yanked: Contains an incorrect dependency on torch


In [8]:
# Third cell - verify installation
import torch
import torchtext
print(f"PyTorch: {torch.__version__}")  # Should show 1.8.0
print(f"TorchText: {torchtext.__version__}")  # Should show 0.9.0

PyTorch: 2.0.0+cpu
TorchText: 0.15.0


In [10]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.datasets import Multi30k
from torchtext.vocab import build_vocab_from_iterator
from torchtext.data.utils import get_tokenizer

from torch.utils.data import DataLoader

import numpy as np
import spacy
import random

from torch.utils.tensorboard import SummaryWriter


In [14]:
!python -m spacy download de_core_news_sm


     ---------------------------------------- 0.0/14.6 MB ? eta -:--:--
      --------------------------------------- 0.3/14.6 MB ? eta -:--:--
     -- ------------------------------------- 1.0/14.6 MB 3.9 MB/s eta 0:00:04
     ----- ---------------------------------- 1.8/14.6 MB 3.5 MB/s eta 0:00:04
     ------- -------------------------------- 2.6/14.6 MB 3.7 MB/s eta 0:00:04
     --------- ------------------------------ 3.4/14.6 MB 3.8 MB/s eta 0:00:03
     ----------- ---------------------------- 4.2/14.6 MB 3.8 MB/s eta 0:00:03
     ------------- -------------------------- 5.0/14.6 MB 3.8 MB/s eta 0:00:03
     --------------- ------------------------ 5.8/14.6 MB 3.8 MB/s eta 0:00:03
     ----------------- ---------------------- 6.6/14.6 MB 3.8 MB/s eta 0:00:03
     -------------------- ------------------- 7.3/14.6 MB 3.8 MB/s eta 0:00:02
     ---------------------- ----------------- 8.1/14.6 MB 3.8 MB/s eta 0:00:02
     ------------------------ --------------- 8.9/14.6 MB 3.8 MB/s

In [15]:
!python -m spacy download en_core_web_sm


     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
      --------------------------------------- 0.3/12.8 MB ? eta -:--:--
     ---- ----------------------------------- 1.3/12.8 MB 4.2 MB/s eta 0:00:03
     ------ --------------------------------- 2.1/12.8 MB 3.9 MB/s eta 0:00:03
     --------- ------------------------------ 2.9/12.8 MB 4.0 MB/s eta 0:00:03
     ----------- ---------------------------- 3.7/12.8 MB 4.0 MB/s eta 0:00:03
     ------------- -------------------------- 4.5/12.8 MB 3.9 MB/s eta 0:00:03
     ----------------- ---------------------- 5.5/12.8 MB 3.9 MB/s eta 0:00:02
     ------------------- -------------------- 6.3/12.8 MB 3.9 MB/s eta 0:00:02
     ---------------------- ----------------- 7.1/12.8 MB 3.9 MB/s eta 0:00:02
     ------------------------ --------------- 7.9/12.8 MB 3.9 MB/s eta 0:00:02
     --------------------------- ------------ 8.7/12.8 MB 3.9 MB/s eta 0:00:02
     ----------------------------- ---------- 9.4/12.8 MB 3.9 MB/s

In [ ]:
spacy_ger = spacy.load('de_core_news_sm')
spacy_en = spacy.load('en_core_web_sm')


def tokenizer_de(text):
    return [tok.text for tok in spacy_ger.tokenizer(text)]


def tokenizer_en(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]

text = "Hallo, wie geht es dir?"
print(tokenizer_de(text))



german = Field(tokenize = tokenizer_de,
                         lower = True,
                         init_token = '<sos>',
                         eos_token='<eos>')

english = Field(tokenize = tokenizer_en,
                         lower = True,
                         init_token = '<sos>',
                         eos_token='<eos>')



train_data, valid_data, test_data = Multi30k.splits(exts=('.de', '.en'),
                                                    fields=(german, english))


german.build_vocab(train_data, max_size = 10000, min_freq = 2)
english.build_vocab(train_data, max_size = 10000, min_freq = 2)

['Hallo', ',', 'wie', 'geht', 'es', 'dir', '?']


In [21]:
class Encoder(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, num_layers, dropout_value):
        super(Encoder,self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.dropout = nn.dropout(dropout_value)
        self.embedding = nn.Embedding(input_size, embedding_size)
        self.rnn = nn.LSTM(embedding_size, hidden_size, num_layers, dropout=dropout_value)


    def forward(self, x):
        # x shape: (seq_length, N)

        embedding = self.dropout(self.embedding(x))
        #embedding shape: (seq_length, N, embedding_size)

        
        outputs, (hidden, cell) = self.rnn(embedding)

        return hidden, cell

### Hidden size of the Encoder and Decoder must be the same

In [ ]:
class Decoder(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, output_size, num_layers, dropout_value):
        super(Decoder,self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.dropout = nn.Dropout(dropout_value)
        self.embedding = nn.Embedding(input_size, embedding_size)
        self.rnn = nn.LSTM(embedding_size, hidden_size, num_layers, dropout=dropout_value)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden, cell):
        #x shape: (N) -> we want (1, N)
        x = x.unsqueeze(0)

        embedding = self.dropout(self.embedding(x))
        #embedding shape: (1, N, embedding_size)

        outputs, (hidden, cell) = self.rnn(embedding, (hidden, cell))
        #outputs shape: (1, N, hidden_size)
        predictions = self.fc(outputs)
        #predictions shape: (1, N, output_size)
        predictions = predictions.squeeze(0)
        #predictions shape: (N, output_size)
        return predictions, hidden, cell


In [ ]:
class Seq2Seq(nn.Module):
    pass